In this post, I will use the logit regression method to predict default rate.   

# We should have a varibale that show having complete loan or not

### Data
To answer the question, I will analyze loan data for 5000 banking system customers in a Middle Eastern country. 

In [29]:
import pandas as pd
import numpy as np

In [30]:
data = pd.read_csv("C:\\Users\\rtava\\OneDrive\\Documents\\Git\\MLProject\\loans_clean.csv")

### Varibales

|Variable name |Description| 
|-----|:-----|
|Jack |this si a lign ts lsdj sldjf sldfjk sldfkj sldjfl sjdflj sldfj sldkfj slkjd f  |
|Susan|Female|

# Second

## Thsi is True

# Third